# Configurations

In [1]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd

Test_file     = './Dataset/test_66516Ee.csv'
Test_file     = './Dataset/tst_renewal_predicted.csv'
Test_file     = '/Users/dninb/Downloads/MA_Hack/tst_renewal_predicted_GBT2.csv'

# Test_dtype = {'id':np.str, 'perc_premium_paid_by_cash_credit':np.float32, 
# 'age_in_days':np.float32, 
# 'Income':np.float32,
# 'Count_3-6_months_late':np.float32, 
# 'Count_6-12_months_late':np.float32,
# 'Count_more_than_12_months_late':np.float32, 
# 'application_underwriting_score':np.float32,
# 'no_of_premiums_paid':np.float32, 
# 'sourcing_channel':np.str, 
# 'residence_area_type':np.str,
# 'premium':np.int}

Test_dtype = {'id':np.str, 
              'premium':np.int, 
              'renewal':np.float32,
              'incentives':np.float32
             }



## Loading dataset

In [2]:
# Loading dataset
tst_origin = pd.read_csv(Test_file, dtype=Test_dtype, na_values='')

display(tst_origin.head())

#tst_data = tst_origin[['id', 'premium']].copy()
tst_data = tst_origin

#Check the shape of each dataset
print(tst_data.shape)

,id,premium,renewal,improvement,incentives
0,649,3300,0.994348,0.0,0.0
1,81136,11700,0.985195,0.0,0.0
2,70762,11700,0.873101,0.0,0.0
3,53935,5400,0.977759,0.0,0.0
4,15476,9600,0.962916,0.0,0.0


(34224, 5)


### Arbitrarily put renewal probabilities for each policy

In [3]:
#pd_renewal = pd.DataFrame(np.random.rand(len(tst_data),1), columns=['renewal'])
#pd_incentives = pd.DataFrame(np.zeros((len(tst_data), 1)), columns=['incentives'], dtype=np.float32)
#pd_improve = pd.DataFrame(np.zeros((len(tst_data), 1)), columns=['improvement'], dtype=np.float32)
#tst_data2 = pd.concat([tst_data, pd_renewal, pd_improve, pd_incentives], axis=1)

tst_data2 = tst_data
display(tst_data2.head())
tst_data2.shape

,id,premium,renewal,improvement,incentives
0,649,3300,0.994348,0.0,0.0
1,81136,11700,0.985195,0.0,0.0
2,70762,11700,0.873101,0.0,0.0
3,53935,5400,0.977759,0.0,0.0
4,15476,9600,0.962916,0.0,0.0


(34224, 5)

### Build equations

In [4]:
import math

def inc_to_eff(inc):
    return 10*(1-np.exp(-1.0*inc/400.0))

def eff_to_imp(eff):
    return 20*(1-np.exp(-1.0*eff/5.0))/100

# def derivative(inc, delta):
#     y1 = eff_to_imp(inc_to_eff(inc))
#     y2 = eff_to_imp(inc_to_eff(inc+delta))
#     return (y2-y1)/delta

def rev_policy(renewal, premium, inc):
    #print('--', renewal, eff_to_imp(inc_to_eff(inc)), premium, inc )
    return (renewal + renewal*eff_to_imp(inc_to_eff(inc))) * premium - inc

def derivatives(renewal, premium, inc, delta=1):
    y1 = rev_policy(renewal, premium, inc)
    y2 = rev_policy(renewal, premium, inc+delta)
    return (y2-y1)/delta

def total_net_rev(dataset):
    return dataset['']


In [ ]:
print(inc_to_eff(1000))
print(eff_to_imp(10))
print(eff_to_imp(inc_to_eff(100)))
print(derivatives(0.5, 10000, 100, 1))
print(derivatives(0.5, 1000, 100, 1))
print(derivatives(0.8, 10000, 100, 1))
print(derivatives(0.5, 10000, 10, 1))


### Ok, Let's find optimal solution!!!!

In [5]:
import time

#unit_inc = 100
learning_rate = 1.0
delta = 1
MAX_ITER = 1000000
t1 = time.time()

itr = 0
prev_total_net_rev = 0
total_net_rev = 0

while True:
    if itr > 2  and total_net_rev-prev_total_net_rev < 1:
        print('Total net revenue is below 1!')
        break
    if itr > MAX_ITER:
        print('Maximum iteration reached.')
        break
    result = derivatives(tst_data2['renewal'], tst_data2['premium'], tst_data2['incentives'], delta)
    #print(result.head())
    cand_idx, deriv = result.idxmax(), result.max()
    #data.loc[data.bidder == 'parakeet2004', 'bidderrate'] = 100
    tst_data2.loc[cand_idx, 'incentives'] = tst_data2.loc[cand_idx, 'incentives'] + learning_rate * deriv
    tst_data2.loc[cand_idx, 'improvement'] = eff_to_imp(inc_to_eff(tst_data2.loc[cand_idx, 'incentives']))
    
    prev_total_net_rev = total_net_rev
    total_net_rev = (tst_data2['renewal'] + tst_data2['renewal']*tst_data2['improvement']) * tst_data2['premium'] - tst_data2['incentives']
    total_net_rev = np.sum(total_net_rev)
    
    if itr%2000 == 0:
        print('%d\t%d\t%13.3f\t%13.3f\t%13.3f' %(itr, cand_idx, deriv, total_net_rev, total_net_rev-prev_total_net_rev))
    
    itr += 1

print(round(time.time() - t1), 'seconds.')

tst_data2[['id', 'renewal', 'incentives']].to_csv('./Dataset/output_submission.csv', index=False)
#np.exp(1/tst_data2['premium'])

0	26990	       58.522	351536696.575	351536696.575
2000	25878	       29.133	354112482.459	      763.690
4000	18034	       24.099	355363918.230	      531.973
6000	32616	       21.015	356308771.605	      409.011
8000	20159	       19.672	357091765.921	      362.214
10000	33148	       18.478	357764461.413	      319.110
12000	231	       17.503	358368669.340	      290.284
14000	30718	       16.600	358908352.177	      259.281
16000	25459	       16.056	359411978.669	      243.467
18000	3636	       15.352	359877304.352	      225.023
20000	8385	       14.700	360300752.780	      205.030
22000	2876	       14.399	360704794.870	      197.240
24000	16431	       13.933	361088847.521	      185.571
26000	33223	       13.441	361449051.853	      174.293
28000	10970	       13.083	361782288.722	      163.318
30000	17093	       12.832	362103671.965	      157.653
32000	16284	       12.609	362411996.353	      152.136
34000	2882	       12.428	362712640.769	      149.221
36000	23726	       12.050	363000283.906	  

## Save output file for submission

In [ ]:
display(tst_data2)

In [ ]:
tst_data2[['id', 'renewal', 'incentives']].to_csv('./Dataset/output_submission_LR.csv', index=False)